In [5]:
from typing import Tuple, Optional

from xgboost.spark import SparkXGBClassifier
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler

# Dataset reading and preparation 

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
path_to_data = "../../../dataset_offline/ft2model/"
path_to_data = "../../../../dataset_offline/ft2model/"

targets_df = spark.read.parquet(path_to_data + "target_pandas.parquet")
df = spark.read.parquet(path_to_data + "filtered_features.parquet").cache()

23/11/13 14:32:01 WARN CacheManager: Asked to cache already cached data.


In [8]:
df = df.dropDuplicates(subset=['ID_CLIENTE_BIC'])
df = df.join(targets_df, on="ID_CLIENTE_BIC", how="left")

df.show()

+--------------+-------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------+--------------------------------------------------+-------------------------------------------------+----------------------------------------+-----------------------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+-------------------------------------------+-------------------------------------------------+----------------------------------------------------+--------------------------------------------+----------------------------------------------------+-----------------------------------------

In [9]:
df = df.select("somma_entrate_entrate_regolari__number_peaks__n_3", "somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10", "ID_CLIENTE_BIC", "TARGET")
# df = df.select("somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10", "ID_CLIENTE_BIC", "TARGET")

In [10]:
# Un-comment for multiclass problem

# df = df.withColumn(
#     "TARGET",
#     F.when(
#         F.col("ID_CLIENTE_BIC") < 40000000,
#         2
#     ).otherwise(F.col("TARGET"))
# )

In [11]:
def get_features_cols_name(df: DataFrame, excluded_cols_name: list[str]) -> list[str]:
    return [
        col_name for col_name in df.columns if col_name not in excluded_cols_name
    ]

In [12]:
features_cols_name = get_features_cols_name(df, excluded_cols_name=["ID_CLIENTE_BIC", "TARGET"])
len(features_cols_name)

2

# Dataset preparation

## Splitting Dataset

In [13]:
def train_test_split(df: DataFrame, test_size=0.3):
    return df.randomSplit(weights=[1-test_size, test_size])

In [14]:
train_df, test_df = train_test_split(df)

## Categorical encoding

In [15]:
from pyspark.ml.feature import StringIndexer, StringIndexerModel, OneHotEncoder, OneHotEncoderModel
from pyspark.sql.types import StringType
from typing import Union


def _cols_name_encode(cols_name: list[str]) -> list[str]:
    return [col_name + "_encoded" for col_name in cols_name]

def _cols_replace(df, temp_cols_name, cols_name):
    df = df.drop(*cols_name)
    for temp_col_name, col_name in zip(temp_cols_name, cols_name):
        df = df.withColumnRenamed(temp_col_name, col_name)
    
    return df

def _compute_cols_encoding(df: DataFrame, cols_name: list[str], order: str = "ascending") -> dict[str, list[Union[str, int, float]]]:
    if order == "ascending":
        ascending = True
    elif order == "descending":
        ascending = False
    else:
        raise ValueError('order param should be "ascending" or "descending"')
    
    cols_encoding = {}
    for col_name in cols_name:
        categ_options_row = df.select(col_name).distinct().orderBy(col_name, ascending=ascending).collect()
        for categ_option_row in categ_options_row:
            categ_option = categ_option_row[col_name]
            if col_name not in cols_encoding:
                cols_encoding[col_name] = [categ_option]
            else:
                cols_encoding[col_name].append(categ_option)

    return cols_encoding

### Label Encoding

In [16]:
def label_encode(df, cols_name: list[str], string_indexer_model: Optional[StringIndexerModel] = None, order: str = "ascending") -> Tuple[DataFrame, StringIndexerModel, dict[str, list[Union[str, int, float]]]]:
    cols_encoding = _compute_cols_encoding(df=df, cols_name=cols_name, order="ascending")
    temp_cols_name = _cols_name_encode(cols_name)

    if not string_indexer_model:
        # frequencyDesc, frequencyAsc, alphabetDesc, alphabetAsc
        if order == "ascending":
            stringOrderType = "frequencyAsc"
        elif order == "descending":
            stringOrderType = "frequencyDesc"
        else:
            raise ValueError('order param should be "ascending" or "descending"')
        
        string_indexer_model = StringIndexer(inputCols=cols_name, outputCols=temp_cols_name, stringOrderType=stringOrderType).fit(df)

    df = string_indexer_model.transform(df)
    df = _cols_replace(df, temp_cols_name, cols_name)

    return df, string_indexer_model, cols_encoding

### One-Hot-Encoding

In [17]:
def one_hot_encode(
        df: DataFrame, 
        cols_name: list[str],
        one_hot_encoder_model: Optional[OneHotEncoderModel] = None,
        string_indexer_model: Optional[StringIndexerModel] = None,
    ) -> Tuple[DataFrame, OneHotEncoderModel, Optional[StringIndexerModel], dict[str, list[Union[str, int, float]]]]:

    df, string_indexer_model, cols_encoding = label_encode(df, cols_name, string_indexer_model)

    temp_cols_name = _cols_name_encode(cols_name)

    if not one_hot_encoder_model:
        one_hot_encoder_model = OneHotEncoder(inputCols=cols_name, outputCols=temp_cols_name).fit(df)

    df = one_hot_encoder_model.transform(df)
    df = _cols_replace(df, temp_cols_name, cols_name)

    return df, one_hot_encoder_model, string_indexer_model, cols_encoding

### Example

In [18]:
# encoding_df = spark.createDataFrame(data=df.select("somma_uscite_altre_spese__number_peaks__n_5").take(5)).withColumnRenamed("somma_uscite_altre_spese__number_peaks__n_5", "categ_feature")
encoding_expample_df = spark.createDataFrame([("a",), ("b",), ("a",), ("c",)], ["categ_feature"])
encoding_expample_df.show()

+-------------+
|categ_feature|
+-------------+
|            a|
|            b|
|            a|
|            c|
+-------------+



In [19]:
test = spark.createDataFrame([(1,), (1,), (6,), (3,)], ["categ_feature"])
one_hot_encoder_model = OneHotEncoder(inputCols=["categ_feature"], outputCols=["temp_cols_name_encoded"]).fit(test)
one_hot_encoder_model.transform(test).show()

+-------------+----------------------+
|categ_feature|temp_cols_name_encoded|
+-------------+----------------------+
|            1|         (6,[1],[1.0])|
|            1|         (6,[1],[1.0])|
|            6|             (6,[],[])|
|            3|         (6,[3],[1.0])|
+-------------+----------------------+



In [20]:
test = spark.createDataFrame([(2,), (0,), (6,), (4,)], ["categ_feature"])
test_result, _, _, labels = one_hot_encode(test, cols_name=["categ_feature"])
test.show()
test_result.show()

+-------------+
|categ_feature|
+-------------+
|            2|
|            0|
|            6|
|            4|
+-------------+

+-------------+
|categ_feature|
+-------------+
|(3,[1],[1.0])|
|(3,[0],[1.0])|
|    (3,[],[])|
|(3,[2],[1.0])|
+-------------+



In [21]:
test = spark.createDataFrame([("a",), ("c",), ("d",)], ["categ_feature"])
test_result, _, _, labels = one_hot_encode(test, cols_name=["categ_feature"])
test.show()
test_result.show()

+-------------+
|categ_feature|
+-------------+
|            a|
|            c|
|            d|
+-------------+

+-------------+
|categ_feature|
+-------------+
|(2,[0],[1.0])|
|(2,[1],[1.0])|
|    (2,[],[])|
+-------------+



## Vector assembling

In [22]:
from pyspark.ml.linalg import DenseVector, SparseVector

def vector_assemble(
    df: DataFrame, features_cols_name: list[str], output_col_name="features", cols_encoding: Optional[dict[str, list[Union[str, int, float]]]] = None
) -> Tuple[DataFrame, list[str]]:
    vectorAssembler = VectorAssembler(
        inputCols=features_cols_name,
        outputCol=output_col_name,
        handleInvalid="keep",
    )

    new_features_cols_name = []
    for feature_col_name in features_cols_name:
        feature_first_element = df.select(feature_col_name).first()[feature_col_name] # type: ignore
        if isinstance(feature_first_element, (DenseVector, SparseVector)):
            vector_len = len(feature_first_element) + 1
            for idx in range(0, vector_len):
                if cols_encoding is not None and feature_col_name in cols_encoding and vector_len == len(cols_encoding[feature_col_name]):
                    new_features_cols_name.append(f"{feature_col_name}_{cols_encoding[feature_col_name][idx] }")
                else:
                    new_features_cols_name.append(f"{feature_col_name}_{idx}")
        else:
            new_features_cols_name.append(feature_col_name)

    df = vectorAssembler.transform(df)

    return df, new_features_cols_name

In [23]:
train_df, _, _, cols_encoding = one_hot_encode(train_df, cols_name=["somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10"])
test_df, _, _, cols_encoding = one_hot_encode(test_df, cols_name=["somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10"])

In [24]:
preprocessed_train_df, cols_encoding = vector_assemble(train_df, features_cols_name=features_cols_name, cols_encoding=cols_encoding)
preprocessed_test_df, _ = vector_assemble(test_df, features_cols_name=features_cols_name)

In [25]:
cols_encoding

['somma_entrate_entrate_regolari__number_peaks__n_3',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_0.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_1.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_2.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_3.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_4.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_5.0',
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_6.0']

# Model training and predicting

In [26]:
from pyspark.ml import Transformer
from xgboost.spark.core import _SparkXGBEstimator

def get_estimator(features_col_name: str = "features", label_col_name: str = "label"):
    return SparkXGBClassifier(features_col=features_col_name, label_col=label_col_name, enable_sparse_data_optim=True, missing=0.0)

def train(df: DataFrame, estimator: _SparkXGBEstimator) -> Transformer:
    return estimator.fit(df)

def predict(df: DataFrame, model: Transformer) -> DataFrame:
    return model.transform(df)

In [27]:
estimator = get_estimator(label_col_name="TARGET")
model = train(preprocessed_train_df, estimator)

result_df = predict(preprocessed_test_df, model)
result_df.show()

2023-11-13 14:32:09,897 INFO XGBoost-PySpark: _fit Running xgboost-2.0.0 on 1 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': 0.0}
[14:32:12] task 0 got new rank 0                                    (0 + 1) / 1]
/Users/vaccaro/anaconda3/envs/transactionsanalysis/lib/python3.11/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/vaccaro/anaconda3/envs/transactionsanalysis/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
2023-11-13 14:32:13,426 INFO XGBoost-PySpark: _fit Finished xgboost training!   


+-------------------------------------------------+--------------+------+-----------------------------------------------------------+-------------+--------------------+----------+--------------------+
|somma_entrate_entrate_regolari__number_peaks__n_3|ID_CLIENTE_BIC|TARGET|somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10|     features|       rawPrediction|prediction|         probability|
+-------------------------------------------------+--------------+------+-----------------------------------------------------------+-------------+--------------------+----------+--------------------+
|                                              0.0|      41637577|     0|                                                  (6,[],[])|    (7,[],[])|[1.83254241943359...|       0.0|[0.86206430196762...|
|                                              0.0|      43748110|     1|                                                  (6,[],[])|    (7,[],[])|[1.83254241943359...|       0.0|[0.86206430196762

INFO:XGBoost-PySpark:Do the inference on the CPUs
/Users/vaccaro/anaconda3/envs/transactionsanalysis/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [55]:
model.

Param(parent='SparkXGBClassifier_9bb7a7840326', name='eval_metric', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier.fit() for this param eval_metric')

# Evalutation

In [28]:
from pyspark.ml.evaluation import Evaluator, BinaryClassificationEvaluator, MulticlassClassificationEvaluator

def get_evaluator(metric: str, prediction_col_name: str = "prediction", rawPredictionCol: str = "rawPrediction", label_col_name: str = "label") -> Evaluator:
    if metric in [
        "f1", 
        "accuracy", 
        "weightedPrecision", 
        "weightedRecall", 
        "weightedTruePositiveRate", 
        "weightedFalsePositiveRate", 
        "weightedFMeasure", 
        "truePositiveRateByLabel", 
        "falsePositiveRateByLabel", 
        "precisionByLabel", 
        "recallByLabel", 
        "fMeasureByLabel", 
        "logLoss", 
        "hammingLoss"]:
        return MulticlassClassificationEvaluator(metricName=metric, predictionCol=prediction_col_name, labelCol=label_col_name) # type: ignore
    elif metric in ["areaUnderROC", "areaUnderPR"]:
        return BinaryClassificationEvaluator(metricName=metric, rawPredictionCol=rawPredictionCol, labelCol=label_col_name) # type: ignore
    else:
        raise ValueError("Metric not supported")

In [29]:
evaluator = get_evaluator(metric="f1", label_col_name="TARGET")

## Without Cross Validation

In [30]:
def evaluate(df: DataFrame, evaluator: Evaluator) -> float:
    return evaluator.evaluate(df)

evaluate(result_df, evaluator)

2023-11-13 14:32:14,587 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
/Users/vaccaro/anaconda3/envs/transactionsanalysis/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


0.8383285894406134

## With Cross Validation

In [31]:
# from pyspark.ml.tuning import CrossValidator

# def evaluate_cv(df: DataFrame, estimator: _SparkXGBEstimator, evaluator: Evaluator, num_folds: float = 3) -> float:
#     cv = CrossValidator(estimator=estimator, estimatorParamMaps=[None], evaluator=evaluator, parallelism=1, numFolds=num_folds) # type: ignore
#     model_cv = cv.fit(df)

#     return model_cv.avgMetrics[0]

# evaluate_cv(preprocessed_train_df, estimator, evaluator)

# Features importance

In [32]:
def get_features_importance(
    model: Transformer, cols_encoding: list[str]
) -> dict[str, float]:
    features_ids_and_importances = model.get_feature_importances(importance_type="weight") # type: ignore

    # if len(features_ids_and_importances) > len(features_cols_name):
    #     raise ValueError("features_cols_name has not the right number of elements")
    
    features_names_and_importances = {}
    for idx, feature_col_name in enumerate(cols_encoding):
        feature_id = f"f{idx}"
        if feature_id in features_ids_and_importances:
            features_names_and_importances[feature_col_name] = features_ids_and_importances[feature_id]
        else:
            features_names_and_importances[feature_col_name] = 0.0


    return features_names_and_importances

In [33]:
ft_imp = get_features_importance(model, cols_encoding)
ft_imp

{'somma_entrate_entrate_regolari__number_peaks__n_3': 982.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_0.0': 0.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_1.0': 91.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_2.0': 109.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_3.0': 91.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_4.0': 141.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_5.0': 89.0,
 'somma_entrate_scambio_soldi_tra_privati__number_peaks__n_10_6.0': 0.0}

# Tuning Parametri

https://spark.apache.org/docs/latest/ml-tuning.html#cross-validation

In [53]:
from typing import Union, Tuple
from pyspark.ml import Model

ParamValue = Union[str, int, float]
Params = dict[str, Union[list[ParamValue], ParamValue]]
BestParams = dict[str, ParamValue]

def tune_parameters_cv(df: DataFrame, params: Params, estimator: _SparkXGBEstimator, evaluator: Evaluator, num_folds: int = 3) -> Tuple[BestParams, Model]:
    grid = ParamGridBuilder()
    for param_name, param_value in params.items():
        if isinstance(param_value, list):
            grid = grid.addGrid(estimator.getParam(param_name), param_value)
        else:
            grid = grid.baseOn((estimator.getParam(param_name), param_value))
    grid = grid.build()

    print(grid)
    
    cv = CrossValidator(estimator=estimator, estimatorParamMaps=grid, evaluator=evaluator, parallelism=1, numFolds=num_folds)
    cv_model = cv.fit(df)
    best_model = cv_model.bestModel

    best_params = {}
    for param_name in params.keys():
        best_params[param_name] = best_model.getOrDefault(param=best_model.getParam(paramName=param_name))

    return best_params, best_model

tune_parameters_cv(df=preprocessed_train_df, params={"max_depth": [2, 3]}, estimator=estimator, evaluator=evaluator)

SyntaxError: invalid syntax (178491644.py, line 27)

In [39]:
import itertools

def tune_parameters(train_df: DataFrame, val_df: DataFrame, params: Params, estimator: _SparkXGBEstimator, evaluator: Evaluator) -> Tuple[BestParams, Model]:
    params_temp = {}
    for key, value in params.items():
        if isinstance(value, list):
            params_temp[key] = value
        else:
            params_temp[key] = [value]  
    params = params_temp
        
    keys, values = zip(*params.items())
    params_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]

    best_score = -1
    best_model = None
    best_params = None
    for params in params_grid:
        model = fit(train_df, estimator)
        score = score(val_df, evaluator)

        if score > best_score:
            best_score = score
            best_model = model
            best_params = params
    
    return best_params, best_model # type: disable

tune_parameters(train_df=preprocessed_train_df, val_df=preprocessed_test_df, params={"max_depth": [2, 3], "n_estimators": [100, 200]}, estimator=estimator, evaluator=evaluator)

[{Param(parent='SparkXGBClassifier_9bb7a7840326', name='max_depth', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param max_depth'): 2, Param(parent='SparkXGBClassifier_9bb7a7840326', name='n_estimators', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param n_estimators'): 100}, {Param(parent='SparkXGBClassifier_9bb7a7840326', name='max_depth', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param max_depth'): 2, Param(parent='SparkXGBClassifier_9bb7a7840326', name='n_estimators', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param n_estimators'): 200}, {Param(parent='SparkXGBClassifier_9bb7a7840326', name='max_depth', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param max_depth'): 3, Param(parent='SparkXGBClassifier_9bb7a7840326', name='n_estimators', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param n_estimators'): 100}, {Param(parent='SparkXGBClassifier

In [52]:


params = {"max_depth": [2, 3], "n_estimators": [100, 200], "ciao": 10}
params_temp = {}
for key, value in params.items():
    if isinstance(value, list):
        params_temp[key] = value
    else:
        params_temp[key] = [value]  
params = params_temp


keys, values = zip(*params.items())
params_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]
params_grid

[{'max_depth': 2, 'n_estimators': 100, 'ciao': 10},
 {'max_depth': 2, 'n_estimators': 200, 'ciao': 10},
 {'max_depth': 3, 'n_estimators': 100, 'ciao': 10},
 {'max_depth': 3, 'n_estimators': 200, 'ciao': 10}]

In [50]:
params = {"max_depth": [2, 3], "n_estimators": [100, 200], "ciao": 10}
params_temp = {}
for key, value in params.items():
    if isinstance(value, list):
        params_temp[key] = value
    else:
        params_temp[key] = [value]  
params = params_temp

params

{'max_depth': [2, 3], 'n_estimators': [100, 200], 'ciao': [10]}